In [1]:
import re

with open("day04.input") as file:
    data = file.read()

passports = [dict(re.findall(r"(\w+):([\w#]+)", item)) for item in re.split("\n\n", data)]

len(passports)

255

In [2]:
passports[:2]

[{'hgt': '176cm',
  'iyr': '2013',
  'hcl': '#fffffd',
  'ecl': 'amb',
  'byr': '2000',
  'eyr': '2034',
  'cid': '89',
  'pid': '934693255'},
 {'hcl': '#b5c3db',
  'ecl': 'grn',
  'hgt': '155cm',
  'pid': '#baec97',
  'iyr': '2017',
  'byr': '1939',
  'eyr': '2020'}]

# Part 1

In [3]:
required_fields = ("byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid")

valid_passports = 0
for passport in passports:
    for field in required_fields:
        if field not in passport.keys():
            break
    else:
        valid_passports += 1

valid_passports

230

# Part 2

In [4]:
from typing import Literal

from pydantic import BaseModel, conint, constr, validator

In [5]:
class Passport(BaseModel):
    byr: conint(ge=1920, le=2002)
    iyr: conint(ge=2010, le=2020)
    eyr: conint(ge=2020, le=2030)
    hgt: str
    hcl: constr(regex=r"^#[0-9a-f]{6}$")
    ecl: Literal["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
    pid: constr(regex=r"^\d{9}$")

    @validator("hgt", pre=True)
    def valid_height(cls, value: str):
        if match := re.match("^(\d+)(cm|in)", value):
            height, unit = match.groups()
            if unit == "cm":
                assert 150 <= int(height) <= 193
            elif unit == "in":
                assert 59 <= int(height) <= 76
            return value
        return ValueError

In [6]:
valid_passports = 0
for passport in passports:
    try:
        Passport(**passport)
        valid_passports += 1
    except ValueError as e:
        pass

valid_passports

156